# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [ ]:
# imports
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [ ]:
!ollama pull llama3.2

In [ ]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENROUTER_API_KEY')
BASE_URL = 'https://openrouter.ai/api/v1'

if api_key and api_key.startswith('sk-or-v1-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
openai = OpenAI(
    api_key=api_key,
    base_url=BASE_URL)

OLLAMA_BASE_URL = "http://localhost:11434/v1"

ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')   

In [ ]:
# Prompts with single shot prompting
system_prompt = """
You are a professional AI coding tutor.
When given a programming question, provide a comprehensive, step-by-step explanation that is clear, instructive, and designed for learners seeking a deep understanding.
Break down concepts, highlight key ideas, use relevant code examples, and point out common mistakes or misconceptions.
Maintain a friendly, expert tone.

Example

Question: What is a Python list comprehension and how does it work?
Explanation:
A list comprehension is a concise way to create lists in Python. It lets you build a new list by applying an expression to each item in an iterable (like a list or range), optionally filtering with a condition. For example, suppose you want a list of squares for numbers 1 through 5:

squares = [x**2 for x in range(1, 6)]

Here, x**2 is the expression, and for x in range(1, 6) iterates over the numbers.
You can also add a condition—for instance, get squares of only even numbers:

even_squares = [x**2 for x in range(1, 6) if x % 2 == 0]
"""

user_prompt = """
I have a coding question and would like a detailed, step-by-step explanation that’s clear and includes code examples. Please highlight important concepts, common mistakes, and give me an expert yet friendly answer.
"""


In [ ]:
question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

def stream_tutor(model, backend):
    if backend == "openai":
        chat_api = openai.chat.completions
    elif backend == "ollama":
        chat_api = ollama.chat.completions
    else:
        raise ValueError("Unsupported backend: choose 'openai' or 'ollama'.")

    stream = chat_api.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt + question}
        ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)        

In [ ]:
# Get gpt-4o-mini to answer, with streaming
stream_tutor('gpt-4.1-mini',"openai")

In [ ]:
# Get Llama 3.2 to answer
stream_tutor('llama3.2',"ollama")